# Imports

In [1]:
import numpy as np
from scipy.optimize import minimize 
import random

# Create random Reachable sets

In [2]:
c = 5 # cache capacity
k = 5 # each user should be in at most K reachable sets
m = 2 # at most M items can be initially recommended to each user
number_of_items = 10
number_of_users = 12
items= np.array(range(0,number_of_items)) # videos
users = np.array(range(0,number_of_users)) # users
# initialize x0 for the optimization
x = np.full(len(items),0.5)
y = np.full((len(items), len(users)),0.5)
x0 = np.concatenate([x.flatten(), y.flatten()])
# Random Reachable sets
reachable_sets = np.array(np.array([[np.array(random.sample(list(users), random.choice(users))) for user in users] for i in items]))
I = np.zeros((len(items), len(users), len(users)))
for i in items:
    for u in users:
        for v in reachable_sets[i][u]:
            I[i][u][v] = 1

# Objective function

In [3]:
def objective(z):
    x = z[:len(items)]
    y = z[len(items):].reshape(len(items),len(users))
    
    #The first term in (1) counts the users to which an item is 
    #initially recommended and the users in the ensuing reachable sets
    #first_term = sum([(1+len(reachable_sets[i][u])) * y[i][u] for u in users])
    #The second term in (1) is due to the overlap of reachable sets Riu,Riv
    #second_term = 0.5*sum([sum([len(set(reachable_sets[i][u])&set(reachable_sets[i][v]))*y[i][u]*y[i][v] for v in users if v!=u]) for u in users])
    return -sum([x[i]*(sum([(1+len(reachable_sets[i][u])) * y[i][u] for u in users]) - 0.5*sum([sum([len(set(reachable_sets[i][u])&set(reachable_sets[i][v]))*y[i][u]*y[i][v] for v in users if v!=u]) for u in users])) for i in items])

# Constraint (2) is the cache capacity constraint

In [4]:
def constraint2(z):  
    x = z[:len(items)]
    
    return sum([x[i] for i in items]) - c

# Constraint (3) says that at most M items can be initially recommended to each user

In [5]:
def constraint3(z):  
    y = z[len(items):].reshape(len(items),len(users))
    
    return [(-sum([y[i][u] for i in items]))+m for u in users]

# Constraint (4) says that each user should be in at most K reachable sets these capture content consumption, attention span or time constraints

In [6]:
def constraint4(z):  
    y = z[len(items):].reshape(len(items),len(users))
    
    return [-sum([sum([I[i][u][v]*y[i][u] for u in users]) for i in items])+k for u in users]

# Solve

In [7]:
con2 = {'type': 'eq', 'fun': constraint2} 
con3 = {'type': 'ineq', 'fun': constraint3} 
con4 = {'type': 'ineq', 'fun': constraint4} 
cons = ([con2, con3, con4])
# Bounds
b = (0.0,1.0)
bnds = [b for i in range(len(x0))]

# show initial objective
print('Initial Objective: ' + str(-1*objective(x0)))

solution = minimize(objective, x0, method='SLSQP', bounds=bnds, constraints=cons)

Initial Objective: -25.25


In [8]:
x = solution.x

# show final objective
print('Final Objective: ' + str(-1*objective(x)))

Final Objective: 81.99999999936897


In [9]:
solution

     fun: -81.99999999936897
     jac: array([-16.54260731,  -0.27305222, -15.99246979, -17.55488682,
       -15.        , -16.91003704,  -0.04125977,  -0.2991333 ,
        -0.23250294,  -0.21064854,  -3.        ,   3.        ,
        -2.        ,   5.        ,  -2.        ,   2.        ,
        -1.        ,   5.        ,  -1.        ,   5.        ,
         1.        ,   3.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   4.        ,  -1.        ,
        -3.        ,   2.        ,  -1.        ,   4.        ,
        -2.        ,   0.        ,   3.        ,  -2.        ,
         4.        ,   1.        ,   1.        ,  -1.        ,
         3.55488586,  -2.44511414,  -0.44511414,   4.55488586,
         6.55488586,  -4.        ,  -1.        ,  -1.        ,
        -1.        ,   1.        ,  -1.        ,   0.        ,
         2.     